# Naive Bayes

In [1]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

#Retorna la ruta completa del archivo y su contenido (sin la cabecera)
def readFiles(path):
    #os.walk itera sobre todos los archivos de una carpeta
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                #se salta todo el encabezado y empieza a leer en el 
                #primera linea vacía
                elif line == '\n':
                    inBody = True
            f.close()
            #Join all items in a list into a string, using a \n as separator
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

#Un DataFrame con dos columnas, uno contiene el contenido del correo y el otro el tipo (spam o ham) 
data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('./datos/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('./datos/emails/ham', 'ham'))


In [2]:
data.head()

,class,message
./datos/emails/spam\00001.7848dde101aa985090474a91ec93fcf0,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
./datos/emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
./datos/emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
./datos/emails/spam\00004.eac8de8d759b7e74154f142194282724,spam,##############################################...
./datos/emails/spam\00005.57696a39d7d84318ce497886896bf90d,spam,I thought you might like these:\n\n1) Slim Dow...


Usamos CountVectorizer para dividir cada mensaje en un listado de palabras. Luego lo introducimos en el clasificador MultinomialNB. Llamando a fit() obtenemos el filtro de spam entrenado.

In [7]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)
#Palabras y su frecuencia en cada documento
#print(counts)
#El vocabulario
#print(vectorizer.vocabulary_.items())
classifier = MultinomialNB()
targets = data['class'].values

classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

Probamos con algunos ejemplos:

In [11]:
examples = ['buy viagra now.', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'],
      dtype='<U4')

Actividades:
- Probar correos spam nuevos y ver que resultado arroja
- Hacer train/test para probar el modelo